In [40]:
class ImageData:
    def __init__(self, path, ymin, xmin, ymax, xmax):
        self.path = path
        self.xmin = xmin      
        self.ymin = ymin        
        self.xmax = xmax        
        self.ymax = ymax

In [44]:
from PIL import Image
import csv

imagepath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DatasætDeeplearning/license_plates_detection_train/'
csvpath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DatasætDeeplearning/license_plates_detection_train.csv'

def read_image_data():
    file = open(csvpath)
    csvreader = csv.reader(file)
    rows = []
    next(csvreader)
    for row in csvreader:
            id = ImageData(row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]))
            rows.append(id)
    file.close()
    return rows


In [58]:
import os

newPath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DatasætDeeplearning/croppedImages/'
os.makedirs(newPath, exist_ok=True)

imageData = read_image_data()

for id in imageData:
    im = Image.open(imagepath + id.path)
    im = im.crop( (id.xmin, id.ymin, id.xmax, id.ymax) )
    im.save(newPath + id.path) 
    im.close()